In [1]:
import random
import numpy as np
import os

import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import cv2

import zipfile


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [2]:
CFG = {
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':42
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
#저장된 이미지 쌍을 동시에 로드 

class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.damage_files = sorted(os.listdir(damage_dir))
        self.origin_files = sorted(os.listdir(origin_dir))

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img}

In [6]:
# 경로 설정
origin_dir = '/kaggle/input/imagedataset/train_gt'  # 원본 이미지 폴더 경로
damage_dir = '/kaggle/input/imagedataset/train_input'  # 손상된 이미지 폴더 경로
test_dir = '/kaggle/input/imagedataset/test_input'     # test 이미지 폴더 경로

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 데이터셋 및 DataLoader 생성
dataset = CustomDataset(damage_dir=damage_dir, origin_dir=origin_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=1)

In [11]:
import torch
import torch.nn as nn

# 간단한 U-Net Generator
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNetGenerator, self).__init__()

        def down_block(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, kernel_size=4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        def up_block(in_feat, out_feat):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, kernel_size=4, stride=2, padding=1),
                      nn.ReLU(inplace=True)]
            return nn.Sequential(*layers)

        # 다운샘플링 단계를 줄이고 채널 수를 줄임
        self.down1 = down_block(in_channels, 32, normalize=False)
        self.down2 = down_block(32, 64)
        self.down3 = down_block(64, 128)
        self.down4 = down_block(128, 256, normalize=False)

        # 업샘플링 단계
        self.up1 = up_block(256, 128)
        self.up2 = up_block(256, 64)
        self.up3 = up_block(128, 32)
        self.up4 = nn.Sequential(
            nn.ConvTranspose2d(64, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        # 다운샘플링
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)

        # 업샘플링 (skip connection 포함)
        u1 = self.up1(d4)
        u2 = self.up2(torch.cat([u1, d3], 1))
        u3 = self.up3(torch.cat([u2, d2], 1))
        u4 = self.up4(torch.cat([u3, d1], 1))

        return u4

# 간단한 PatchGAN Discriminator
class PatchGANDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(PatchGANDiscriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, kernel_size=4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        # 디스크리미네이터의 레이어 수를 줄임
        self.model = nn.Sequential(
            discriminator_block(in_channels * 2, 32, normalization=False),
            discriminator_block(32, 64),
            discriminator_block(64, 128),
            nn.Conv2d(128, 1, kernel_size=4, padding=1)
        )

    def forward(self, img_A, img_B):
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)


In [14]:
# 가중치 초기화 함수
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [16]:
device

device(type='cuda', index=0)

In [17]:
# 모델 저장을 위한 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

best_loss = float("inf")
lambda_pixel = 100  # 픽셀 손실에 대한 가중치
        
# Generator와 Discriminator 초기화
generator = UNetGenerator()
generator = generator.to(device)
discriminator = PatchGANDiscriminator()
discriminator = discriminator.to(device)

generator.apply(weights_init_normal).to(device)
discriminator.apply(weights_init_normal).to(device)

# 손실 함수 및 옵티마이저 설정
criterion_GAN = nn.MSELoss()
criterion_pixelwise = nn.L1Loss()

optimizer_G = optim.Adam(generator.parameters(), lr = CFG["LEARNING_RATE"])
optimizer_D = optim.Adam(discriminator.parameters(), lr = CFG["LEARNING_RATE"]) 

# 학습
for epoch in range(1, CFG['EPOCHS'] + 1):
    for i, batch in enumerate(dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        # Generator 훈련
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake).to(device))
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()

        # Discriminator 훈련
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real).to(device))
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake).to(device))
        loss_D = 0.5 * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()

        # Discriminator 정확도 계산
        real_accuracy = torch.mean((pred_real > 0.5).float())  # 진짜 이미지를 진짜로 예측한 비율
        fake_accuracy = torch.mean((pred_fake < 0.5).float())  # 가짜 이미지를 가짜로 예측한 비율
        accuracy = 0.5 * (real_accuracy + fake_accuracy)       # 진짜와 가짜에 대한 평균 정확도

        # 진행 상황 출력
        print(f"[Epoch {epoch}/{CFG['EPOCHS']}] [Batch {i}/{len(dataloader)}] "
              f"[D loss: {loss_D.item()}] [G loss: {loss_G.item()}] [D accuracy: {accuracy.item() * 100:.2f}%]")

        # 현재 에포크에서의 손실이 best_loss보다 작으면 모델 저장
        if loss_G.item() < best_loss:
            best_loss = loss_G.item()
            torch.save(generator.state_dict(), os.path.join(model_save_dir, "best_generator.pth"))
            torch.save(discriminator.state_dict(), os.path.join(model_save_dir, "best_discriminator.pth"))
            print(f"Best model saved at epoch {epoch}, batch {i} with G loss: {loss_G.item()} and D loss: {loss_D.item()}")

[Epoch 1/3] [Batch 0/1851] [D loss: 0.993576169013977] [G loss: 41.52439880371094]
Best model saved at epoch 1, batch 0 with G loss: 41.52439880371094 and D loss: 0.993576169013977
[Epoch 1/3] [Batch 1/1851] [D loss: 1.2126719951629639] [G loss: 40.62755584716797]
Best model saved at epoch 1, batch 1 with G loss: 40.62755584716797 and D loss: 1.2126719951629639
[Epoch 1/3] [Batch 2/1851] [D loss: 0.6424400210380554] [G loss: 37.65669250488281]
Best model saved at epoch 1, batch 2 with G loss: 37.65669250488281 and D loss: 0.6424400210380554
[Epoch 1/3] [Batch 3/1851] [D loss: 0.4848468005657196] [G loss: 37.340370178222656]
Best model saved at epoch 1, batch 3 with G loss: 37.340370178222656 and D loss: 0.4848468005657196
[Epoch 1/3] [Batch 4/1851] [D loss: 0.4826509952545166] [G loss: 43.57659149169922]
[Epoch 1/3] [Batch 5/1851] [D loss: 0.4317202568054199] [G loss: 38.75200271606445]
[Epoch 1/3] [Batch 6/1851] [D loss: 0.3685450553894043] [G loss: 34.025508880615234]
Best model save

In [18]:
# 저장할 디렉토리 설정
submission_dir = "./submission"
os.makedirs(submission_dir, exist_ok=True)

# 이미지 로드 및 전처리
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원을 추가합니다.
    return image

# 모델 경로 설정
generator_path = os.path.join(model_save_dir, "best_generator.pth")

# 모델 로드 및 설정
model = UNetGenerator().to(device)
model.load_state_dict(torch.load(generator_path))
model.eval()

# 파일 리스트 불러오기
test_images = sorted(os.listdir(test_dir))

# 모든 테스트 이미지에 대해 추론 수행
for image_name in test_images:
    test_image_path = os.path.join(test_dir, image_name)

    # 손상된 테스트 이미지 로드 및 전처리
    test_image = load_image(test_image_path).to(device)

    with torch.no_grad():
        # 모델로 예측
        pred_image = model(test_image)
        pred_image = pred_image.cpu().squeeze(0)  # 배치 차원 제거
        pred_image = pred_image * 0.5 + 0.5  # 역정규화
        pred_image = pred_image.numpy().transpose(1, 2, 0)  # HWC로 변경
        pred_image = (pred_image * 255).astype('uint8')  # 0-255 범위로 변환
        
        # 예측된 이미지를 실제 이미지와 같은 512x512로 리사이즈
        pred_image_resized = cv2.resize(pred_image, (512, 512), interpolation=cv2.INTER_LINEAR)

    # 결과 이미지 저장
    output_path = os.path.join(submission_dir, image_name)
    cv2.imwrite(output_path, cv2.cvtColor(pred_image_resized, cv2.COLOR_RGB2BGR))    
    
print(f"Saved all images")

/tmp/ipykernel_30/2583963943.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(generator_path))


Saved all images


In [19]:
# 저장된 결과 이미지를 ZIP 파일로 압축
zip_filename = "submission.zip"
with zipfile.ZipFile(zip_filename, 'w') as submission_zip:
    for image_name in test_images:
        image_path = os.path.join(submission_dir, image_name)
        submission_zip.write(image_path, arcname=image_name)

print(f"All images saved in {zip_filename}")

All images saved in submission.zip


In [21]:
from IPython.display import FileLink
FileLink(r'submission.zip')

/kaggle/working/submission.zip

In [23]:
!git clone https://github.com/saic-mdal/lama.git

Cloning into 'lama'...
remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 469 (delta 148), reused 109 (delta 108), pack-reused 200 (from 1)
Receiving objects: 100% (469/469), 8.83 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (186/186), done.


In [33]:
# requirements.txt 파일 내용 확인
requirements_path = '/kaggle/input/lama-model/requirements.txt'
with open(requirements_path, 'r') as f:
    requirements = f.readlines()

# 필요한 패키지 설치
for requirement in requirements:
    package = requirement.strip()
    if package:
        !pip install {package}


  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6345 sha256=1b7146f7973acb4d3686cbf53f04bf186c9e4474707c02a4f835ed04e1b7808f
  Stored in directory: /root/.cache/pip/wheels/fd/d2/35/4c11d19a72280492846f4c4df975311a2bac475e8021f86c1d
Successfully built easydict
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 46.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-image: filename=scikit_image-0.17.2-cp310-cp310-linux_x86_64.whl size=12863605 sha256=7b6dc3f6a07249a1922a52418d949c4b910824badcd63b340953168d7846c7b3
  Stored in directory: /root/.cache/pip/wheels/30/fc/f4/10c6987c3a9cb937913fa19f8290e91e6f66df2440c2af6130
Successfully built scikit-image
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.23.2
    Uninstalling scikit-image-0.23.2:
      Successfully uninstalled scikit-image-0.23.2
ERROR: pip's dependency resolver does not cur

In [42]:
# !pip install --upgrade Cython
# !pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [46]:
# !pip install scikit-learn==0.24.2 --no-build-isolation    //쓸모없음 에러발생함
# !pip uninstall -y scikit-image
# !pip install scikit-image
!pip uninstall -y albumentations
!pip install albumentations==0.4.6

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Found existing installation: albumentations 0.5.2
Uninstalling albumentations-0.5.2:
  Successfully uninstalled albumentations-0.5.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65153 sha256=e854078163666b440d083cf2ba8eada2fd6fcc213f5833a88b4eade68316fad8
  Stored in directory: /root/.cache/pip/wheels/f9/d7/0c/6ed42fd872f7d1af78b25045f8b16be330f2c70ae72c83e37d
Successfully built albumentations


In [50]:
# '/kaggle/input/lama-model' 경로를 모두 제거


In [54]:
sys.path = [p for p in sys.path if p != '/kaggle/input/lama-model/saicinpainting' and p != '/kaggle/input/lama-model']

In [55]:
sys.path 

['/kaggle/lib/kagglegym',
 '/kaggle/lib',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/root/.local/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages',
 '/root/src/BigQuery_Helper']

In [56]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# /kaggle/input/lama-model 경로를 Python 경로에 추가하여 saicinpainting을 인식하게 함

# 중복 없이 경로 추가
# sys.path = [p for p in sys.path if p != '/kaggle/input/lama-model']
if '/kaggle/input/lama-model' not in sys.path:
    sys.path.append('/kaggle/input/lama-model')

In [62]:
sys.path

['/kaggle/lib/kagglegym',
 '/kaggle/lib',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/root/.local/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages',
 '/root/src/BigQuery_Helper',
 '/kaggle/input/lama-model']

In [63]:
# saicinpainting 디렉토리를 Python 경로에 추가
saicinpainting_path = '/kaggle/input/lama-model/saicinpainting'
if saicinpainting_path not in sys.path:
    sys.path.append(saicinpainting_path)

# 현재 sys.path 출력하여 경로가 추가되었는지 확인
print(sys.path)

['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper', '/kaggle/input/lama-model', '/kaggle/input/lama-model/saicinpainting']


In [65]:
# import sys
# from albumentations.core.transforms_interface import DualTransform

# # DualIAATransform을 직접 정의하여 albumentations 모듈에 추가
# class DualIAATransform(DualTransform):
#     def __init__(self, always_apply=False, p=0.5):
#         super(DualIAATransform, self).__init__(always_apply, p)

#     def apply(self, img, **params):
#         return img

#     def apply_to_mask(self, img, **params):
#         return img

# # to_tuple 함수 정의
# def to_tuple(param, low=None):
#     if isinstance(param, tuple):
#         return param
#     return (param, param) if low is None else (low, param)

# # albumentations 모듈에 DualIAATransform과 to_tuple이 포함되도록 추가
# sys.modules['albumentations'].DualIAATransform = DualIAATransform
# sys.modules['albumentations'].to_tuple = to_tuple

In [ ]:
# 모델 저장을 위한 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

best_loss = float("inf")
lambda_pixel = 100  # 픽셀 손실에 대한 가중치

In [69]:
from saicinpainting.training.trainers.default import DefaultInpaintingTrainingModule as DefaultTrainer
from saicinpainting.training.data.datasets import InpaintingTrainDataset as ImageInpaintingDataset
from saicinpainting.training.modules.base import SimpleMultiStepGenerator as InpaintingModel

In [78]:
from saicinpainting.training.modules.base import SimpleMultiStepGenerator

# SimpleMultiStepGenerator를 상속하여 forward 메서드만 수정
class SimpleMultiStepGeneratorModified(SimpleMultiStepGenerator):
    def forward(self, x):
        cur_in = x
        for step in self.steps:
            cur_in = step(cur_in)  # 이전 출력과 연결하지 않고 현재 출력을 다음 입력으로 사용
        return cur_in  # 최종 출력만 반환


In [79]:
# Generator를 SimpleMultiStepGeneratorModified로 초기화
generator_steps = [
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # 중간 채널 수 유지
    nn.ReLU(),
    nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)  # 최종 출력 채널을 3으로 설정
]
generator = SimpleMultiStepGeneratorModified(generator_steps)
generator = generator.to(device)

In [75]:
# PatchGAN Discriminator 초기화
discriminator = PatchGANDiscriminator()
discriminator = discriminator.to(device)

# 가중치 초기화
generator.apply(weights_init_normal).to(device)
discriminator.apply(weights_init_normal).to(device)

# 손실 함수 및 옵티마이저 설정
criterion_GAN = nn.MSELoss()
criterion_pixelwise = nn.L1Loss()

optimizer_G = optim.Adam(generator.parameters(), lr=CFG["LEARNING_RATE"])
optimizer_D = optim.Adam(discriminator.parameters(), lr=CFG["LEARNING_RATE"]) 

In [80]:
# 학습
for epoch in range(1, CFG['EPOCHS'] + 1):
    for i, batch in enumerate(dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        # Generator 훈련
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake).to(device))
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()

        # Discriminator 훈련
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real).to(device))
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake).to(device))
        loss_D = 0.5 * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()

        # Discriminator 정확도 계산
        real_accuracy = torch.mean((pred_real > 0.5).float())  # 진짜 이미지를 진짜로 예측한 비율
        fake_accuracy = torch.mean((pred_fake < 0.5).float())  # 가짜 이미지를 가짜로 예측한 비율
        accuracy = 0.5 * (real_accuracy + fake_accuracy)       # 진짜와 가짜에 대한 평균 정확도

        # 진행 상황 출력
        print(f"[Epoch {epoch}/{CFG['EPOCHS']}] [Batch {i}/{len(dataloader)}] "
              f"[D loss: {loss_D.item()}] [G loss: {loss_G.item()}] [D accuracy: {accuracy.item() * 100:.2f}%]")
        
        # 현재 에포크에서의 손실이 best_loss보다 작으면 모델 저장
        if loss_G.item() < best_loss:
            best_loss = loss_G.item()
            torch.save(generator.state_dict(), os.path.join(model_save_dir, "best_generator.pth"))
            torch.save(discriminator.state_dict(), os.path.join(model_save_dir, "best_discriminator.pth"))
            print(f"Best model saved at epoch {epoch}, batch {i} with G loss: {loss_G.item()} and D loss: {loss_D.item()}")


[Epoch 1/3] [Batch 0/1851] [D loss: 0.895781397819519] [G loss: 47.76409912109375]
Best model saved at epoch 1, batch 0 with G loss: 47.76409912109375 and D loss: 0.895781397819519
[Epoch 1/3] [Batch 1/1851] [D loss: 0.7947466373443604] [G loss: 54.122535705566406]
[Epoch 1/3] [Batch 2/1851] [D loss: 0.48124465346336365] [G loss: 37.47613525390625]
Best model saved at epoch 1, batch 2 with G loss: 37.47613525390625 and D loss: 0.48124465346336365
[Epoch 1/3] [Batch 3/1851] [D loss: 0.37855473160743713] [G loss: 47.040748596191406]
[Epoch 1/3] [Batch 4/1851] [D loss: 0.35326772928237915] [G loss: 41.342342376708984]
[Epoch 1/3] [Batch 5/1851] [D loss: 0.36269518733024597] [G loss: 43.26234436035156]
[Epoch 1/3] [Batch 6/1851] [D loss: 0.36004504561424255] [G loss: 46.96315383911133]
[Epoch 1/3] [Batch 7/1851] [D loss: 0.368450403213501] [G loss: 41.2018928527832]
[Epoch 1/3] [Batch 8/1851] [D loss: 0.3566524386405945] [G loss: 42.35653305053711]
[Epoch 1/3] [Batch 9/1851] [D loss: 0.329

In [85]:
# 저장할 디렉토리 설정
submission_dir = "./submission2"
os.makedirs(submission_dir, exist_ok=True)

# 이미지 로드 및 전처리
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원을 추가합니다.
    return image

# 모델 경로 설정
generator_path = os.path.join(model_save_dir, "best_generator.pth")

# 모델 로드 및 설정
model = SimpleMultiStepGeneratorModified(generator_steps).to(device)
model.load_state_dict(torch.load(generator_path))
model.eval()
# 파일 리스트 불러오기
test_images = sorted(os.listdir(test_dir))

# 모든 테스트 이미지에 대해 추론 수행
for image_name in test_images:
    test_image_path = os.path.join(test_dir, image_name)

    # 손상된 테스트 이미지 로드 및 전처리
    test_image = load_image(test_image_path).to(device)

    with torch.no_grad():
        # 모델로 예측
        pred_image = model(test_image)
        pred_image = pred_image.cpu().squeeze(0)  # 배치 차원 제거
        pred_image = pred_image * 0.5 + 0.5  # 역정규화
        pred_image = pred_image.numpy().transpose(1, 2, 0)  # HWC로 변경
        pred_image = (pred_image * 255).astype('uint8')  # 0-255 범위로 변환
        
        # 예측된 이미지를 실제 이미지와 같은 512x512로 리사이즈
        pred_image_resized = cv2.resize(pred_image, (512, 512), interpolation=cv2.INTER_LINEAR)

    # 결과 이미지 저장
    output_path = os.path.join(submission_dir, image_name)
    cv2.imwrite(output_path, cv2.cvtColor(pred_image_resized, cv2.COLOR_RGB2BGR))    
    
print(f"Saved all images")

/tmp/ipykernel_30/2990091767.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(generator_path))


Saved all images


In [86]:
# 저장된 결과 이미지를 ZIP 파일로 압축
zip_filename = "submission2.zip"
with zipfile.ZipFile(zip_filename, 'w') as submission_zip:
    for image_name in test_images:
        image_path = os.path.join(submission_dir, image_name)
        submission_zip.write(image_path, arcname=image_name)

print(f"All images saved in {zip_filename}")

All images saved in submission2.zip


In [70]:
from lama.model import LaMaModel

ModuleNotFoundError: No module named 'lama.model'

In [ ]:
# 모델 저장을 위한 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

best_loss = float("inf")
lambda_pixel = 100  # 픽셀 손실에 대한 가중치

# Generator와 Discriminator 초기화
generator = lama_model
generator = generator.to(device)

# PatchGAN Discriminator 초기화
discriminator = PatchGANDiscriminator()
discriminator = discriminator.to(device)

# 가중치 초기화
generator.apply(weights_init_normal).to(device)
discriminator.apply(weights_init_normal).to(device)

# 손실 함수 및 옵티마이저 설정
criterion_GAN = nn.MSELoss()
criterion_pixelwise = nn.L1Loss()

optimizer_G = optim.Adam(generator.parameters(), lr=CFG["LEARNING_RATE"])
optimizer_D = optim.Adam(discriminator.parameters(), lr=CFG["LEARNING_RATE"]) 

# 학습
for epoch in range(1, CFG['EPOCHS'] + 1):
    for i, batch in enumerate(dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        # Generator 훈련
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake).to(device))
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()

        # Discriminator 훈련
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real).to(device))
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake).to(device))
        loss_D = 0.5 * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()

        # 진행 상황 출력
        print(f"[Epoch {epoch}/{CFG['EPOCHS']}] [Batch {i}/{len(dataloader)}] [D loss: {loss_D.item()}] [G loss: {loss_G.item()}]")

        # 현재 에포크에서의 손실이 best_loss보다 작으면 모델 저장
        if loss_G.item() < best_loss:
            best_loss = loss_G.item()
            torch.save(generator.state_dict(), os.path.join(model_save_dir, "best_generator.pth"))
            torch.save(discriminator.state_dict(), os.path.join(model_save_dir, "best_discriminator.pth"))
            print(f"Best model saved at epoch {epoch}, batch {i} with G loss: {loss_G.item()} and D loss: {loss_D.item()}")
            
        print(f"Best model saved at epoch {epoch}, batch {i} with G loss: {loss_G.item()} and D loss: {loss_D.item()}")
